# Actions menées en amont
Le fichier .csv présent dans ce projet a été crée à partir d'un fichier XML téléchargé à partir de cette source :

http://redac.univ-tlse2.fr/lexiques/wiktionaryx.html. 

Ce Notebook explique comment le fichier XML, non présent dans le dépot final en raison de sa taille (+300Mb), a été parsé et transformé en fichier CSV.

Le fichier XML recensait plus de 900.000 mots en français (y compris les pluriels, variations orthographiques etc...) avec leurs définitions, leurs traductions en différentes langues, leurs exemples d'utilisation etc. 

Pour des raisons de concision et afin de réduire au mieux la taille du fichier final, ce projet a fait le choix de ne garder que les mots et leurs définitions et de transformer le fichier XML en un fichier CSV, ce qui d'une part le débarasse de toutes les syntaxes balisées et d'autre part permet une meilleur synergie avec le module Pandas pour l'exploitation ultérieure des données.

## Parsing du fichier XML

In [3]:
from bs4 import BeautifulSoup

Une fois le module importé, nous créons un objet Beautifulsoup (instancié ici sous le nom de 'soup') par l'ouverture du fichier 'dico.xml' présent dans le dossier courant. L'objet contiendra la totalité du code XML du fichier `dico.xml`

In [4]:
# Création de l'objet soup (peut prendre plusieurs minutes)
with open("dico.xml", mode='r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, features='xml')

Dans ce fichier XML, chaque mot est contenu dans des balises `<entry>`.

Voici un exemple (tronqué) du rendu XML pour le mot 'manga' :

```
<entry form="manga" pageid="21">
    ...
    <gloss>Bande dessinée japonaise, souvent en noir et blanc et à la pagination élevée.</gloss>
    ...
    <gloss>Style, stéréotypes et règles implicites de la bande dessinée japonaise.</gloss>
    ...
</entry>
```

Comme nous le voyons, le mot 'manga' apparaît dans le paramètre `form` de la balise `<entry>`, et les définitions, pouvant être multiples pour chaque mot, apparaissent entre des balises `<gloss>`.

La première étape consitera donc à parser toutes les balises `<entry>` dans l'objet `soup`.

In [5]:
res = soup.findAll('entry')

L'objet `res` contient toutes les balises `<entry> </entry>` du code XML et toutes les données se trouvant entre celles-ci, notamment les balises `<gloss> </gloss>` qui nous interessent particulièrement.

 Cet objet peut maintenant être itéré, chaque itération correspondra à une nouvelle balise `<entry>` (et donc à un nouveau mot) dans laquelle nous pourront parser nos informations utiles.

 Voici un affichage stylisé des 4 permiers mot du fichier XML grace à des boucles for.   

In [6]:
i = 0

for entry in res:
    print("Mot : ", entry['form'].capitalize())
    gloss = entry.findAll('gloss', string=True, limit=4)

    for idx, definition in enumerate(gloss):
        print(f"{idx+1})", definition.text)
    
    i+=1
    if i == 4:
        break

Mot :  Accueil
1) Cérémonie ou prestation réservée à un nouvel arrivant, consistant généralement à lui souhaiter la bienvenue et à l'aider dans son intégration ou ses démarches.
2) Lieu où sont accueillies les personnes.
3) Faire accueil dans le sens d'accueillir ou héberger.
Mot :  Dictionnaire
1) Ouvrage de référence qui répertorie des mots dans un ordre convenu (alphabétique en général) pour leur associer :
2) une définition, un sens,
3) un ou plusieurs synonymes, antonymes, etc.,
4) une étymologie,
Mot :  Lire
1) Interpréter des informations écrites sous forme de mots ou de dessins sur un support.
2) Suivre des yeux ce qui est écrit ou imprimé, avec la connaissance des sons que les lettres figurent; soit en ne proférant pas les mots, soit en les proférant à haute voix.
3) Comprendre ce qui est écrit ou imprimé dans une langue étrangère.
4) Parcourir des yeux une musique notée, avec la connaissance des sons que les notes figurent et des diverses modifications que ces sons doivent re

Nous remerquons que les mots n'ont pas été classés par ordre alphabétique, mais ceci est un détail qui pourra aisément être réglé une fois le fichier .csv crée.
<!-- blank line -->
----
<!-- blank line -->
C'est à partir de ces mêmes boucles for que nous allons maintenant créer notre fichier .csv

## Création du fichier .csv

In [8]:
import csv

In [54]:
with open('dico.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    colonnes = ["Mot", "Définitions"] # Noms de nos colonnes

    writer.writerow(colonnes) # Ecriture 1ère ligne (nom colonnes)
    row_null = 0

    for entry in res:
        mot = entry['form'].capitalize() # Récupération du mot

        # Récupération des définitions dans une liste
        def_parse = entry.findAll('gloss', string=True) # limit=5
        def_list = []
        for definition in def_parse:
            def_list.append(definition.text)

        # Certains mots exotiques ne possèdent aucune définition,
        # si la liste ne récupére aucune entrée, ou si la 
        # liste ne contient qu'un point le mot ne sera
        # pas inséré au fichier .csv (+4000 mots/+900.000 
        # n'ont aucune définition)
        if len(def_list) > 0 and def_list != ['.']:
            writer.writerow([mot, def_list]) # Ecriture de la ligne
        else: # Aucune définition
            row_null += 1
            continue
    
    print("Total null : ", row_null)

Total null :  4403


## Exploitation du fichier .csv

In [71]:
import pandas as pd

In [72]:
# Création du dataframe Pandas
df = pd.read_csv("dico.csv")

### Rapide exploration

In [73]:
# Affichage des dimensions
df.shape

(973671, 2)

In [74]:
# Affichage des 5 premières lignes 
df.head()

,Mot,Définitions
0,Accueil,"[""Cérémonie ou prestation réservée à un nouvel..."
1,Dictionnaire,['Ouvrage de référence qui répertorie des mots...
2,Lire,['Interpréter des informations écrites sous fo...
3,Encyclopédie,"['Ensemble, enchaînement de toutes les connais..."
4,Manga,"['Bande dessinée japonaise, souvent en noir et..."


In [75]:
# Affichage des 5 dernières lignes 
df.tail()

,Mot,Définitions
973666,Sévice,['mauvais traitement.']
973667,Faire voir,"['Montrer, faire connaître.']"
973668,Laisser voir,"['Montrer, faire connaître.']"
973669,Coronavirus,['Genre de virus à ARN de la famille des coron...
973670,Hangûl,"[""système d'écriture employé en coréen. Exempl..."


### Quelques manipulations
#### Classement par ordre alphabétique
La brève exploration nous a permis de constater que les mots n'étaient pas classés par ordre alphabétique. La méthode `sort_values()` axée sur la colonne 'Mot' nous permettra d'y remedier :

In [76]:
df = df.sort_values('Mot')

In [77]:
# Réinitialisation des index du dataframe pour s'assurer de leur
# linéarité
df = df.reset_index(drop=True)

In [87]:
df.head(10)

,Mot,Définitions
0,&amp;,['variante ortho de et.']
1,&amp;c.,['variante ortho de etc.']
2,'alif,['variante ortho de alif.']
3,'tain,['Forme familière de putain.']
4,'tis,"[""Masculin pluriel de 'ti.""]"
5,'tite,"[""Féminin singulier de 'ti."", ""Féminin singuli..."
6,'tites,"[""Féminin pluriel de 'ti."", ""Féminin pluriel d..."
7,'tits,"[""Masculin pluriel de 'tit.""]"
8,'zza,"[""Dix-septième lettre de l'alphabet arabe : /z..."
9,'álif,['variante ortho de alif.']


#### Afficher les définitions d'un mot

In [79]:
# Définitions d'un mot
D = df.loc[df['Mot'] == 'Manga']['Définitions']
D

444909    ['Bande dessinée japonaise, souvent en noir et...
Name: Définitions, dtype: object

In [80]:
type(D)

pandas.core.series.Series

Cela nous renvoie une Series Pandas avec la valeur de l'index et la liste contenant les définitions, c'est cette dernière qui nous intéresse.

In [81]:
# Récupération de la liste des définitions
D.values

array(["['Bande dessinée japonaise, souvent en noir et blanc et à la pagination élevée.', 'Style, stéréotypes et règles implicites de la bande dessinée japonaise.', 'Peut aussi désigner un dessin animé japonais, mais ce terme est impropre car le terme courant est « anime ».']"],
      dtype=object)

In [82]:
type(D.values)

numpy.ndarray

La valeur est contenue dans un array Numpy (l'importion du module Numpy n'est pour autant pas nécessaire). Nous constatons que l'objet contient notre liste sous forme de chaîne de caractères, la méthode `eval()` permettra de la convertir en liste.

In [83]:
eval(D.values[0])

['Bande dessinée japonaise, souvent en noir et blanc et à la pagination élevée.',
 'Style, stéréotypes et règles implicites de la bande dessinée japonaise.',
 'Peut aussi désigner un dessin animé japonais, mais ce terme est impropre car le terme courant est « anime ».']

Version condensée

In [92]:
mot = "Manga"
D = df.loc[df['Mot'] == mot]['Définitions']
D = eval(D.values[0])

print("Mot : ", mot)
for idx, definition in enumerate(D):
    print(f"{idx+1}) {definition}")

Mot :  Manga
1) Bande dessinée japonaise, souvent en noir et blanc et à la pagination élevée.
2) Style, stéréotypes et règles implicites de la bande dessinée japonaise.
3) Peut aussi désigner un dessin animé japonais, mais ce terme est impropre car le terme courant est « anime ».
